In [1]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, MSADataGrabberCSV, DataGrabberCSV
import pandas as pd
import numpy as np
import requests
from us import states

root = find_repo_root()



In [7]:
# # code for retrieving available variables

# variables_url = 'https://api.census.gov/data/2010/acs/acs5/subject/variables.json'


# response = requests.get(variables_url)
# variables_info = response.json()

# variables_df = pd.DataFrame.from_dict(variables_info['variables'], orient='index').reset_index()

# variables_df.to_excel(f'{root}/data/variables.xlsx', index=False)

In [8]:
print('Warning: The process will take around 15min.')

# S2301_C04_001E Estimate!!Unemployment rate!!Population 16 years and over
variables =  "NAME,S2301_C04_001E"
county_fips = "*" # all counties
tract = "*" # all tracts
api_key = '077d857d6c12d5b9b3aeafa07d2c1916ba12a86c' # private api key required to access the data https://api.census.gov/data/key_signup.html
# year = 2022

interval = list(range(2010, 2023))
dfs = []

for year in interval:
    for x in range(0, len(states.STATES)): # in this call it's not possible to use the '*' wildcard to access all states, so we need to iterate over all states
        fips = states.STATES[x].fips

        url = f'https://api.census.gov/data/{year}/acs/acs5/subject?get={variables}&for=tract:{tract}&in=state:{fips}&in=county:{county_fips}&key={api_key}'

        response = requests.get(url)

        assert response.status_code == 200, 'The data retrieval went wrong'  # 200 means success

        print(f'{fips} fips for year {year} done')

        data = response.json()

        df = pd.DataFrame(data[1:], columns=data[0])
        df['Year'] = year  # Add the year column

        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

01 fips for year 2010 done
02 fips for year 2010 done
04 fips for year 2010 done
05 fips for year 2010 done
06 fips for year 2010 done
08 fips for year 2010 done
09 fips for year 2010 done
10 fips for year 2010 done
12 fips for year 2010 done
13 fips for year 2010 done
15 fips for year 2010 done
16 fips for year 2010 done
17 fips for year 2010 done
18 fips for year 2010 done
19 fips for year 2010 done
20 fips for year 2010 done
21 fips for year 2010 done
22 fips for year 2010 done
23 fips for year 2010 done
24 fips for year 2010 done
25 fips for year 2010 done
26 fips for year 2010 done
27 fips for year 2010 done
28 fips for year 2010 done
29 fips for year 2010 done
30 fips for year 2010 done
31 fips for year 2010 done
32 fips for year 2010 done
33 fips for year 2010 done
34 fips for year 2010 done
35 fips for year 2010 done
36 fips for year 2010 done
37 fips for year 2010 done
38 fips for year 2010 done
39 fips for year 2010 done
40 fips for year 2010 done
41 fips for year 2010 done
4

In [9]:
unemployment_combined = combined_df.copy()

column_mapping = {
    'S2301_C04_001E': 'Value'
}

unemployment_combined.rename(columns=column_mapping, inplace=True)

unemployment_combined.head()

,NAME,Value,state,county,tract,Year
0,"Census Tract 110.01, Etowah County, Alabama",6.1,01,055,011001,2010
1,"Census Tract 110.02, Etowah County, Alabama",10.3,01,055,011002,2010
2,"Census Tract 111, Etowah County, Alabama",4.7,01,055,011100,2010
3,"Census Tract 112, Etowah County, Alabama",17.9,01,055,011200,2010
4,"Census Tract 200, Fayette County, Alabama",4.3,01,057,020000,2010


In [10]:
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

unemployment_combined['GeoFIPS'] = unemployment_combined.apply(lambda row: f"{row['state']}{row['county']}{row['tract']}", axis=1).astype(np.int64)

unemployment_combined.drop(['state', 'county', 'tract'], axis=1, inplace=True)

def parse_geo_name(name):
    if ';' in name:
        parts = name.split(';')
    else:
        parts = name.split(',')

    if len(parts) >= 3:
        county = parts[1].strip().replace(' County', '')
        state_full = parts[2].strip()
        state_abbr = state_abbreviations.get(state_full, state_full)  
        return f"{county}, {state_abbr} (CT)"
    return "Unknown"


unemployment_combined['GeoName'] = unemployment_combined['NAME'].apply(parse_geo_name).astype(str)

assert unemployment_combined[unemployment_combined['GeoName'] == 'Unknown'].shape[0] == 0, 'There are Unknown GeoNames'

unemployment_combined = unemployment_combined.drop(['NAME'], axis=1)

unemployment_combined.sort_values(by=['Year', 'GeoFIPS', 'GeoName'], inplace=True)
unemployment_combined = unemployment_combined[['GeoFIPS', 'GeoName', 'Year', 'Value']].reset_index(drop=True)

unemployment_combined.head()

,GeoFIPS,GeoName,Year,Value
0,1001020100,"Autauga, AL (CT)",2010,3.8
1,1001020200,"Autauga, AL (CT)",2010,7.9
2,1001020300,"Autauga, AL (CT)",2010,5.7
3,1001020400,"Autauga, AL (CT)",2010,4.5
4,1001020500,"Autauga, AL (CT)",2010,3.7


In [11]:
unemployment_combined_pre2020 = unemployment_combined[unemployment_combined['Year'] < 2020].reset_index(drop=True)
unemployment_combined_post2020 = unemployment_combined[unemployment_combined['Year'] >= 2020].reset_index(drop=True)

In [12]:
geo_counts = unemployment_combined_pre2020['GeoFIPS'].value_counts()
geo_in_all_years = geo_counts[geo_counts == geo_counts.max()].index.tolist()
unemployment_combined_pre2020_filtered = unemployment_combined_pre2020[unemployment_combined_pre2020['GeoFIPS'].isin(geo_in_all_years)]
missin_count = unemployment_combined_pre2020['GeoFIPS'].nunique() - unemployment_combined_pre2020_filtered['GeoFIPS'].nunique()

print(f" {missin_count} GeoFIPS values were removed from the pre-2020 data")


 51 GeoFIPS values were removed from the pre-2020 data


In [13]:
geo_counts = unemployment_combined_post2020['GeoFIPS'].value_counts()
geo_in_all_years = geo_counts[geo_counts == geo_counts.max()].index.tolist()
unemployment_combined_post2020_filtered = unemployment_combined_post2020[unemployment_combined_post2020['GeoFIPS'].isin(geo_in_all_years)]
missin_count = unemployment_combined_post2020['GeoFIPS'].nunique() - unemployment_combined_post2020_filtered['GeoFIPS'].nunique()

print(f" {missin_count} GeoFIPS values were removed from the post-2020 data")

 1767 GeoFIPS values were removed from the post-2020 data


In [14]:
unemployment_combined_post2020_filtered_wide = unemployment_combined_post2020_filtered.pivot(index=['GeoFIPS', 'GeoName'], columns='Year', values='Value')
unemployment_combined_post2020_filtered_wide = unemployment_combined_post2020_filtered_wide.reset_index()
unemployment_combined_post2020_filtered_wide.columns.name = None
unemployment_combined_post2020_filtered_wide.head()

,GeoFIPS,GeoName,2020,2021,2022
0,1001020100,"Autauga, AL (CT)",2.1,2.5,2.6
1,1001020200,"Autauga, AL (CT)",4.0,4.5,5.5
2,1001020300,"Autauga, AL (CT)",2.7,2.4,1.9
3,1001020400,"Autauga, AL (CT)",2.4,2.2,0.9
4,1001020501,"Autauga, AL (CT)",1.0,1.0,0.6


In [15]:
unemployment_combined_pre2020_filtered_wide = unemployment_combined_pre2020_filtered.pivot(index=['GeoFIPS', 'GeoName'], columns='Year', values='Value')
unemployment_combined_pre2020_filtered_wide = unemployment_combined_pre2020_filtered_wide.reset_index()
unemployment_combined_pre2020_filtered_wide.columns.name = None
unemployment_combined_pre2020_filtered_wide.head()

,GeoFIPS,GeoName,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1001020100,"Autauga, AL (CT)",3.8,2.9,3.6,2.8,4.9,5.4,4.2,4.6,3.6,2.9
1,1001020200,"Autauga, AL (CT)",7.9,12.4,13.2,15.3,15.5,13.3,7.8,3.4,4.5,4.4
2,1001020300,"Autauga, AL (CT)",5.7,4.5,2.5,3.6,4.3,6.2,5.1,4.7,3.3,3.5
3,1001020400,"Autauga, AL (CT)",4.5,6.0,10.1,11.5,10.3,10.8,9.3,6.1,2.9,3.2
4,1001020500,"Autauga, AL (CT)",3.7,5.0,3.8,5.3,3.5,4.2,1.4,2.3,2.1,3.1


In [16]:
unemployment_combined_pre2020_filtered_wide = unemployment_combined_pre2020_filtered_wide.dropna(how='any')
unemployment_combined_post2020_filtered_wide = unemployment_combined_post2020_filtered_wide.dropna(how='any')

columns_to_convert = unemployment_combined_pre2020_filtered_wide.columns[2:]  
unemployment_combined_pre2020_filtered_wide[columns_to_convert] = unemployment_combined_pre2020_filtered_wide[columns_to_convert].astype(float)

columns_to_convert = unemployment_combined_post2020_filtered_wide.columns[2:]  
unemployment_combined_post2020_filtered_wide[columns_to_convert] = unemployment_combined_post2020_filtered_wide[columns_to_convert].astype(float)

print(f"Pre-2020 data shape: {unemployment_combined_pre2020_filtered_wide.shape}")
print(f"Post-2020 data shape: {unemployment_combined_post2020_filtered_wide.shape}")


Pre-2020 data shape: (72798, 12)
Post-2020 data shape: (83325, 5)


In [17]:
unemployment_combined_pre2020_filtered_wide.to_csv(f"{root}/data/raw/unemployment_pre2020_ct.csv", index=False)
unemployment_combined_post2020_filtered_wide.to_csv(f"{root}/data/raw/unemployment_post2020_ct.csv", index=False)